In [1]:
import plotly.graph_objects as go
import networkx as nx
import numpy as np

from utils import (get_edges, 
                   get_nodes, 
                   get_degree, 
                   build_all_to_all, 
                   build_graph_viz, 
                   build_node_trace, 
                   build_edge_trace)

# Visualize random graph

In [2]:
G = nx.random_geometric_graph(200, 0.125)

In [3]:
edge_x, edge_y = get_edges(G)
node_x, node_y = get_nodes(G)
node_degree, node_text = get_degree(G)

In [4]:
# edge_trace = build_edge_trace(edge_x, edge_y)
# node_trace = build_node_trace(node_x, node_y)
# node_trace.marker.color = node_degree
# node_trace.text = node_text

# fig = build_graph_viz(edge_trace, node_trace)
# fig.show()

# Cooperative and Competitive Homogeneous Agents

In [5]:
G = build_all_to_all(10)

In [6]:
adjacency_matrix = nx.linalg.graphmatrix.adjacency_matrix(G)
print(adjacency_matrix.todense())

[[0 1 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 1 0 1 1 1 1 1 1 1]
 [1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 0 1 1 1 1 1]
 [1 1 1 1 1 0 1 1 1 1]
 [1 1 1 1 1 1 0 1 1 1]
 [1 1 1 1 1 1 1 0 1 1]
 [1 1 1 1 1 1 1 1 0 1]
 [1 1 1 1 1 1 1 1 1 0]]


In [7]:
# resistance term: d_{ij} 
# In the general model, a larger dij implies a greater resistance of agent i to 
# forming a non-neutral opinion about option j

# social term: 
# the  product  of  an attention parameter ui ≥ 0 
# and a saturating function of weighted sums of agent opinions that are available 
# to agent i and influence its opinion  of  option j
# The  social  term  can  also  be  interpreted as an activation term

# The magnitude of Ajl_ik determines the  strength  of  influence  of  agent k’s  
# opinion  about  option l on  agent i’s  opinion  about  option j,  and  the
# sign  of Ajl_ik determines whether this interaction is excitatory (Ajlik>0) 
# orinhibitory (Ajlik<0).

# The input bij ∈ R represents  an  input  signal  from  the environment  
# or  a  bias  or  predisposition  that  directly  affects agent i’s  opinion  of  option j

In [8]:
# eq 5a (z is the relative opinion)
# z_ij_dot = F_ij(Z) - 1/N_o \sum_l F_il(Z)
# F_ij(Z) = -d_ij z_ij + u_i ( S_1(\sum_k Ajj_ik z_kj) + \sum_(l neq j) S_2(\sum_k Ajl_ik z_kl)) + b_ij

In [10]:
def compute_drift(D, u, Z, A, B):
    '''
    computes drift using S_1 = S_2 = tanh
    
    F_ij(Z) = -d_ij z_ij + u_i ( S_1(\sum_k Ajj_ik z_kj) + \sum_(l neq j) S_2(\sum_k Ajl_ik z_kl)) + b_ij
    
    
    Arguments:
    ---------
    D: matrix 
        size - (number of agents x number of options)
        resistance term. In the general model, a larger dij implies a greater resistance of agent i to 
        forming a non-neutral opinion about option j.
    
    u: vector
        len(U) = number of agents
        attention parameter.
        
    Z: matrix
        size - (number of agents x number of options)
        
    A: tuple
        (Intra-agent, same-option coupling: Ajj_ii
         Intra-agent, inter-option coupling: Ajl_ii, j neq l
         Inter-agent, same-option coupling: Ajj_ik, i neq k
         Inter-agent, inter-option coupling: Ajl_ik, i neq k,j neq l)
         
    B: matrix
    '''
    intra_agent_same_option = A[0] # 2D matrix
    intra_agent_inter_option = A[2]
    inter_agent_same_option = A[3] # 3D matrix
    inter_agent_inter_option = A[4]
    
    social_term = compute_social_term(inter_agent_same_option, inter_agent_inter_option, Z)
    
    # F_ij(Z) = -d_ij z_ij + u_i ( S_1(\sum_k Ajj_ik z_kj) + \sum_(l neq j) S_2(\sum_k Ajl_ik z_kl)) + b_ij
    return -D * Z + social_term + B

In [ ]:
def compute_social_term(inter_agent_same_option, inter_agent_inter_option, Z):
    # social term is
    # the  product  of  an attention parameter ui ≥ 0 
    # and a saturating function of weighted sums of agent opinions that are available 
    # to agent i and influence its opinion  of  option j
    
    # \sum_k Ajj_ik z_kj
    weighted_inter_agent_same_option = np.einsum('jik,kj->ji', inter_agent_same_option, Z)
    thresholded_weighted_inter_agent_same_option = np.tanh(weighted_inter_agent_same_option)
    # \sum_k Ajl_ik z_kl
    weighted_inter_agent_inter_option = np.einsum('jlik,kl->jli', inter_agent_inter_option, Z)
    thresh_weighted_inter_agent_inter_option = np.tanh(weighted_inter_agent_inter_option)
    avg_thresh_weighted_inter_agent_inter_option = np.mean(thresh_weighted_inter_agent_inter_option, axis=1)
    
    social_term = u * (thresholded_weighted_inter_agent_same_option + avg_thresh_weighted_inter_agent_inter_option)
    
    return social_term